In [1]:
!pip install konlpy
!pip install pymysql

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.0-cp38-cp38-macosx_10_9_x86_64.whl (381 kB)
  Using cached lxml-4.9.1-cp38-cp38-macosx_10_15_x86_64.whl (4.6 MB)
  Using cached numpy-1.23.1-cp38-cp38-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [1]:
import pandas as pd
import pymysql

conn = pymysql.connect(
                        user    = 'stocks',
                        passwd  = 'Stocks!',
                        host    = "-",
                        port    = 3306,
                        db      = 'Data',
                        charset = 'utf8'
        )

cur = conn.cursor()

cur.execute('''
SELECT DISTINCT * FROM Stock_Price
''')
result = cur.fetchall()

df = pd.DataFrame(result, columns=['stock_id', 'date', 'closing_price', 'market_price', 'high_price', 'low_price'])

df = df.sort_values(ascending=False, by=['stock_id', 'date']).reset_index(drop=True)

df

,stock_id,date,closing_price,market_price,high_price,low_price
0,402340,2022-07-18,40950,40650,41450,40600
1,402340,2022-07-15,40350,40100,40550,39050
2,402340,2022-07-14,39800,40450,40500,39800
3,402340,2022-07-13,40600,39950,41150,39700
4,402340,2022-07-12,39500,39400,39950,39100
...,...,...,...,...,...,...
14995,000060,2022-04-06,47650,46700,48450,46700
14996,000060,2022-04-05,46700,47050,47050,45400
14997,000060,2022-04-04,46950,47650,48800,46600
14998,000060,2022-04-01,47650,47000,48200,46650


In [29]:
cur.execute('''
SELECT DISTINCT stock_id, text, date FROM Stock_News_2
Where date < '2022-07-16 00:00:00'
''')
news = cur.fetchall()
news = pd.DataFrame(news, columns=['stock_id', 'text', 'date'])
news

,stock_id,text,date
0,000060,2500억 모집액 못 채우고 미달\n3년물 2200억에 1710억 모집…5년물 전액...,2022-07-15 19:54:00
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00
2,000060,증시 과매도…低PER 종목 쏟아진다\n30개社 PER 1~2배로 떨어져\n\nPER...,2022-07-13 17:36:00
3,000060,"투자의견 ‘매수’, 목표주가 7만5000원 유지\n사진=연합뉴스\n[파이낸셜뉴스] ...",2022-07-13 08:18:00
4,000060,"금감원, 손보사에 유사암·일반암 진단비 자료 요구\n올해들어 손보사 유사암 경쟁…진...",2022-07-13 06:01:00
...,...,...,...
111682,402340,앱출석 후 회원가입시\n80만명 한정 선착순[헤럴드경제=박이담 기자]국내 최초 가상...,2022-04-01 11:17:00
111683,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00
111684,402340,31일 증권신고서 제출…4월 기관 수요예측\n\n\n[파이낸셜뉴스] SK스퀘어 자회...,2022-04-01 09:00:00
111685,402340,[서울경제]\n\n<유가증권>\n\n▲삼부토건(001470)=제3자 배정증자 방식으...,2022-04-01 07:01:00


In [30]:
import re

def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', d)
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

In [31]:
news['text'] = news.text.apply(clean_text)
news

,stock_id,text,date
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00
...,...,...,...
111682,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00
111683,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00
111684,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00
111685,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00


In [32]:
from konlpy.tag import Kkma
from tqdm import tqdm, tqdm_pandas
tagger = Kkma()

In [33]:
news.loc[(news["text"] == '') | (news["text"] == ' '), 'text'] = None
news.isnull().sum()

stock_id     0
text        68
date         0
dtype: int64

In [34]:
news = news.dropna(axis=0)
news.isnull().sum()

stock_id    0
text        0
date        0
dtype: int64

In [43]:
def token(news):
    return tagger.morphs(news)
tqdm.pandas()
news['token'] = news.text.progress_apply(token)

100%|██████████| 111619/111619 [42:44:26<00:00,  1.38s/it]   
/var/folders/f0/dnr__kl94mdgsv025xlblkmc0000gn/T/ipykernel_1009/2581450601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['token'] = news.text.progress_apply(token)


In [45]:
news.to_csv("./Token.csv", index=True)

In [53]:
from tqdm import tqdm
import datetime
for i in tqdm(range(0,len(df),75)):
  for j in range(75):
    if df.loc[i+j][1] == datetime.date(2022,3,31):
      df.loc[i+j, 'label1'] = -1
      continue
    if df.loc[i+j][3] <= df.loc[i+j][2]:
      df.loc[i+j, 'label1'] = 1
    elif df.loc[i+j][3] > df.loc[i+j][2]:
      df.loc[i+j, 'label1'] = 0

100%|██████████| 200/200 [00:16<00:00, 12.29it/s]


In [54]:
from tqdm import tqdm_pandas, tqdm
import datetime
for i in tqdm(range(0,len(df),75)):
  for j in range(75):
    if df.loc[i+j][1] == datetime.date(2022,3,31):
      df.loc[i+j, 'label2'] = -1
      continue
    if df.loc[i+j][3] < df.loc[i+j+1][2]:
      df.loc[i+j, 'label2'] = 0
    elif df.loc[i+j][3] >= df.loc[i+j+1][2]:
      df.loc[i+j, 'label2'] = 1

100%|██████████| 200/200 [00:20<00:00,  9.80it/s]


In [56]:
df['label1'] = df.label1.astype('int64')
df['label2'] = df.label2.astype('int64')
df

,stock_id,date,closing_price,market_price,high_price,low_price,label1,label2
0,402340,2022-07-18,40950,40650,41450,40600,1,1
1,402340,2022-07-15,40350,40100,40550,39050,1,1
2,402340,2022-07-14,39800,40450,40500,39800,0,0
3,402340,2022-07-13,40600,39950,41150,39700,1,1
4,402340,2022-07-12,39500,39400,39950,39100,1,0
...,...,...,...,...,...,...,...,...
14995,000060,2022-04-06,47650,46700,48450,46700,1,1
14996,000060,2022-04-05,46700,47050,47050,45400,0,1
14997,000060,2022-04-04,46950,47650,48800,46600,0,1
14998,000060,2022-04-01,47650,47000,48200,46650,1,0


In [57]:
news.reset_index(drop=True, inplace=True)
news

,stock_id,text,date,token,label
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"[2500, 억, 모집, 액, 못, 채우, 고, 미달, 3, 년, 물, 2200, ...",1.0
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"[메리츠증권, 은, 14, 일, 아모레, G, 에, 대하, 어, 아모레, 퍼시픽, ...",0.0
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"[증시, 과, 매도, PER, 종목, 쏟아지, ㄴ다, 30, 개, PER, 1, 2...",1.0
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"[투자, 의견, 매수, 목표, 주가, 7, 만, 5000, 원, 유지, 메리츠증권,...",0.0
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"[금감원, 손보사, 에, 유사, 암, 일반, 암, 진단, 비, 자료, 요구, 올해,...",0.0
...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"[앱, 출석, 후, 회원, 가, 입시, 80, 만, 명, 한정, 선착순, 국내, 최...",NaN
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"[SK, 그룹, 투자, 회사, SK, 스퀘어, 의, 연결, 대상, 종속, 회사, 원...",NaN
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"[31, 일, 증권, 신고서, 제출, 4, 월, 기관, 수요, 예측, SK, 스퀘어...",NaN
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"[삼부, 토건, 제, 3, 자, 배정, 증자, 방식, 으로, SK, 에코, 플랜트,...",NaN


In [59]:
import datetime
from datetime import time
from datetime import timedelta


for i in tqdm(range(len(news))):
  if len(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == news.loc[i][2].date())]) == 0:
    for j in range(1, 7):
      if len(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == (news.loc[i][2] + timedelta(days=j)).date())]) == 0:
        continue
      news.loc[i, 'label'] = int(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == (news.loc[i][2] + timedelta(days=j)).date())]['label2'])
    continue
  if news.loc[i][2].weekday() == 4 and news.loc[i][2].time() > datetime.time(18,0):
    for j in range(1, 7):
      if len(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == (news.loc[i][2] + timedelta(days=j)).date())]) == 0:
        continue
      news.loc[i, 'label'] = int(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == (news.loc[i][2] + timedelta(days=j)).date())]['label2'])
  elif news.loc[i][2].weekday() == 4 and news.loc[i][2].time() <= datetime.time(18,0):
    news.loc[i, 'label'] = int(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == news.loc[i][2].date())]['label1'])
  else:
    if news.loc[i][2].time() > datetime.time(18):
      news.loc[i, 'label'] = int(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == news.loc[i][2].date())]['label2'])
    elif news.loc[i][2].time() <= datetime.time(18):
      news.loc[i, 'label'] = int(df.loc[(df["stock_id"] == news.loc[i][0]) & (df["date"] == news.loc[i][2].date())]['label1'])
        

100%|██████████| 111619/111619 [39:54<00:00, 46.61it/s] 


In [60]:
news.to_csv("./Label.csv", index=True)

In [62]:
news['label'] = news.label.astype('int64')
news

/var/folders/f0/dnr__kl94mdgsv025xlblkmc0000gn/T/ipykernel_1009/139120647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['label'] = news.label.astype('int64')


,stock_id,text,date,token,label
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"[2500, 억, 모집, 액, 못, 채우, 고, 미달, 3, 년, 물, 2200, ...",1
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"[메리츠증권, 은, 14, 일, 아모레, G, 에, 대하, 어, 아모레, 퍼시픽, ...",0
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"[증시, 과, 매도, PER, 종목, 쏟아지, ㄴ다, 30, 개, PER, 1, 2...",1
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"[투자, 의견, 매수, 목표, 주가, 7, 만, 5000, 원, 유지, 메리츠증권,...",0
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"[금감원, 손보사, 에, 유사, 암, 일반, 암, 진단, 비, 자료, 요구, 올해,...",0
...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"[앱, 출석, 후, 회원, 가, 입시, 80, 만, 명, 한정, 선착순, 국내, 최...",1
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"[SK, 그룹, 투자, 회사, SK, 스퀘어, 의, 연결, 대상, 종속, 회사, 원...",1
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"[31, 일, 증권, 신고서, 제출, 4, 월, 기관, 수요, 예측, SK, 스퀘어...",1
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"[삼부, 토건, 제, 3, 자, 배정, 증자, 방식, 으로, SK, 에코, 플랜트,...",1


In [7]:
import pandas as pd

news = pd.read_csv("./Label.csv", index_col=0)

In [12]:
news['label'] = news.label.astype('int64')
news['stock_id'] = news.stock_id.astype('str')
news

,stock_id,text,date,token,label
0,60,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"['2500', '억', '모집', '액', '못', '채우', '고', '미달',...",1
1,60,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"['메리츠증권', '은', '14', '일', '아모레', 'G', '에', '대하...",0
2,60,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"['증시', '과', '매도', 'PER', '종목', '쏟아지', 'ㄴ다', '3...",1
3,60,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"['투자', '의견', '매수', '목표', '주가', '7', '만', '5000...",0
4,60,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"['금감원', '손보사', '에', '유사', '암', '일반', '암', '진단'...",0
...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"['앱', '출석', '후', '회원', '가', '입시', '80', '만', '...",1
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"['SK', '그룹', '투자', '회사', 'SK', '스퀘어', '의', '연결...",1
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"['31', '일', '증권', '신고서', '제출', '4', '월', '기관',...",1
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"['삼부', '토건', '제', '3', '자', '배정', '증자', '방식', ...",1


In [13]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111619 entries, 0 to 111618
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   stock_id  111619 non-null  object
 1   text      111619 non-null  object
 2   date      111619 non-null  object
 3   token     111619 non-null  object
 4   label     111619 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 5.1+ MB


In [15]:
news['stock_id'] = news.stock_id.apply(lambda x : x.zfill(6))
news

,stock_id,text,date,token,label
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"['2500', '억', '모집', '액', '못', '채우', '고', '미달',...",1
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"['메리츠증권', '은', '14', '일', '아모레', 'G', '에', '대하...",0
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"['증시', '과', '매도', 'PER', '종목', '쏟아지', 'ㄴ다', '3...",1
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"['투자', '의견', '매수', '목표', '주가', '7', '만', '5000...",0
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"['금감원', '손보사', '에', '유사', '암', '일반', '암', '진단'...",0
...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"['앱', '출석', '후', '회원', '가', '입시', '80', '만', '...",1
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"['SK', '그룹', '투자', '회사', 'SK', '스퀘어', '의', '연결...",1
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"['31', '일', '증권', '신고서', '제출', '4', '월', '기관',...",1
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"['삼부', '토건', '제', '3', '자', '배정', '증자', '방식', ...",1


In [41]:
from tqdm import tqdm

for row in tqdm(news.itertuples()):

    sql = (row[1], row[2], row[3], row[4], row[5])
    cur.execute('INSERT IGNORE INTO Token (stock_id, text, date, token, label) VALUES (%s ,%s, %s, %s, %s)', sql)
    conn.commit()
conn.close()

111619it [18:13:58,  1.70it/s]


In [2]:
sql = 'SELECT stock_id, text, date, token, label FROM Token'
news = pd.read_sql(sql, conn)
news

/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,stock_id,text,date,token,label
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"['2500', '억', '모집', '액', '못', '채우', '고', '미달',...",1
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"['메리츠증권', '은', '14', '일', '아모레', 'G', '에', '대하...",0
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"['증시', '과', '매도', 'PER', '종목', '쏟아지', 'ㄴ다', '3...",1
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"['투자', '의견', '매수', '목표', '주가', '7', '만', '5000...",0
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"['금감원', '손보사', '에', '유사', '암', '일반', '암', '진단'...",0
...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"['앱', '출석', '후', '회원', '가', '입시', '80', '만', '...",1
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"['SK', '그룹', '투자', '회사', 'SK', '스퀘어', '의', '연결...",1
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"['31', '일', '증권', '신고서', '제출', '4', '월', '기관',...",1
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"['삼부', '토건', '제', '3', '자', '배정', '증자', '방식', ...",1


In [5]:
from konlpy.tag import Okt
from tqdm import tqdm
tqdm.pandas()
news['token_okt'] = news.text.progress_apply(Okt().morphs)
news

 70%|███████   | 78345/111619 [6:41:59<146:32:26, 15.85s/it]

: 

In [16]:
from tqdm import tqdm
def stopword(x):
  stopword = [r'상승.*', r'하락.*', r'급등.*', r'급락.*', '상승세', '하락세', '폭등', '폭락', '오름세', '약세', '강세', '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '에', '은', '는', '하']
  return [i for i in x if i not in stopword]
tqdm.pandas()
news["token_okt"] = news.token.progress_apply(stopword)
news

100%|██████████| 111619/111619 [07:21<00:00, 252.57it/s]


,stock_id,text,date,token,label,token_okt
0,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15 19:54:00,"['2500', '억', '모집', '액', '못', '채우', '고', '미달',...",1,"[[, ', 2, 5, 0, 0, ', ,, , ', 억, ', ,, , ', ..."
1,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14 08:36:00,"['메리츠증권', '은', '14', '일', '아모레', 'G', '에', '대하...",0,"[[, ', 메, 리, 츠, 증, 권, ', ,, , ', ', ,, , ', ..."
2,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13 17:36:00,"['증시', '과', '매도', 'PER', '종목', '쏟아지', 'ㄴ다', '3...",1,"[[, ', 증, 시, ', ,, , ', ', ,, , ', 매, ', ,, ..."
3,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13 08:18:00,"['투자', '의견', '매수', '목표', '주가', '7', '만', '5000...",0,"[[, ', 투, ', ,, , ', 견, ', ,, , ', 매, 수, ', ..."
4,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13 06:01:00,"['금감원', '손보사', '에', '유사', '암', '일반', '암', '진단'...",0,"[[, ', 금, 감, 원, ', ,, , ', 손, 보, 사, ', ,, , ..."
...,...,...,...,...,...,...
111614,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01 11:17:00,"['앱', '출석', '후', '회원', '가', '입시', '80', '만', '...",1,"[[, ', 앱, ', ,, , ', 출, 석, ', ,, , ', 후, ', ..."
111615,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01 11:06:00,"['SK', '그룹', '투자', '회사', 'SK', '스퀘어', '의', '연결...",1,"[[, ', S, K, ', ,, , ', 그, 룹, ', ,, , ', 투, ..."
111616,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01 09:00:00,"['31', '일', '증권', '신고서', '제출', '4', '월', '기관',...",1,"[[, ', 3, 1, ', ,, , ', 일, ', ,, , ', 증, 권, ..."
111617,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01 07:01:00,"['삼부', '토건', '제', '3', '자', '배정', '증자', '방식', ...",1,"[[, ', 삼, 부, ', ,, , ', 토, 건, ', ,, , ', 제, ..."


In [ ]:
for row in tqdm(news.itertuples()):

    sql = (row[3], str(row[6]), row[5])
    cur.execute('INSERT IGNORE INTO Okt_Token2 (date, token, label) VALUES (%s, %s, %s)', sql)
    conn.commit()
conn.close()

0it [00:00, ?it/s]


OperationalError: (1054, "Unknown column 'date' in 'field list'")